In [239]:
from lark import Lark
from lark.indenter import Indenter
from lark.tree import Tree
from lark.lexer import Token
import textwrap
import daglet
from ipywidgets import Image
from IPython.display import display
from PIL import Image as PILImage
import io


def get_image_size(image_data):
    file = io.BytesIO(image_data)
    return PILImage.open(file).size


def show_image(image_data, scale=0.7):
    width, height = get_image_size(image_data)
    width *= scale
    height *= scale
    image = Image(value=image_data, width=width, height=height)
    display(image)


syntax = '''
start: WORD "," WORD "!"

    %import common.WORD   // imports from terminal library
    %ignore " "           // Disregard spaces in text
'''

class PythonIndenter(Indenter):
    NL_type = '_NEWLINE'
    OPEN_PAREN_types = ['LPAR', 'LSQB', 'LBRACE']
    CLOSE_PAREN_types = ['RPAR', 'RSQB', 'RBRACE']
    INDENT_type = '_INDENT'
    DEDENT_type = '_DEDENT'
    tab_len = 8

def get_children(node):
    return getattr(node, 'children', [])

def get_label(node):
    #print()
    #print(type(node))
    #print(dir(node))
    if isinstance(node, Tree):
        #print(node.data)
        return node.data
    elif isinstance(node, Token):
        return node.type + '\n' + node.value
    else:
        print(node.type)
        return node.type

def show(parsed):
    image_data = daglet.render(
        [parsed],
        parent_func=get_children,
        vertex_label_func=get_label,
        vertex_color_func=lambda x: '#ffcc00',
    )
    show_image(image_data, 0.32)
    
parser = Lark.open('skylark.lark', parser='lalr', postlex=PythonIndenter(), start='file')



In [247]:
class Return(Exception):
    def __init__(self, value):
        self.value = value


def get_arg_info(param_node, stack):
    name = do_eval(param_node.children[0], stack)
    if '**' in name:
        name = '**' + name
    elif '*' in name:
        name = '*' + name
    if len(param_node.children) > 1:
        has_default = True
        def_value = do_eval(param_node.children[1], stack)
    else:
        has_default = False
        def_value = None
    if '*' in name:
        assert not has_default, 'default value not allowed for {!r}'.format(name)
    return name, has_default, def_value

        
def get_args_info(params_node, stack):
    param_nodes = params_node.children
    args_info = [get_arg_info(x, stack) for x in param_nodes]
    found_kwarg = False
    found_star = False
    found_doublestar = False
    min_arg_count = 0
    max_arg_count = 0
    for name, has_default, _ in args_info:
        if '**' in name:
            bogus = found_double_star
            found_doublestar = True
        elif '*' in name:
            bogus = found_star
            found_star = True
        elif has_default:
            bogus = found_star or found_doublestar
            found_kwarg = True
            max_arg_count += 1
        else:
            bogus = found_kwarg or found_star or found_doublestar
            min_arg_count += 1
            max_arg_count += 1
        assert not bogus, 'argument {!r} not allowed after wildcard args'
    if found_star:
        max_arg_count = None
    return args_info, min_arg_count, max_arg_count, found_star, found_doublestar


def get_stack_item(key, stack):
    for frame in reversed(stack):
        if key in frame:
            return frame[key]
    assert 0, 'Invalid identifier: {!r}'.format(key)


def do_eval(node, stack, as_ref=False):
    if isinstance(node, Token):
        value = node.value
    elif isinstance(node, Tree):
        if node.data == 'string':
            value = node.children[0].value[1:-1]
        elif node.data == 'expression':
            assert len(node.children) == 1, 'tuples not supported yet'
            value = do_eval(node.children[0], stack, as_ref)
        elif node.data in ['file', 'simple_stmt', 'suite']:
            for child_node in node.children:
                do_eval(child_node, stack)
            value = None
        elif node.data in ['arguments']:
            value = [do_eval(x, stack) for x in node.children]
        elif node.data in ['statement', 'operand', 'small_stmt', 'test', 'expr_stmt', 'number', 'string', 'identifier']:
            assert len(node.children) == 1
            value = do_eval(node.children[0], stack, as_ref)
        elif node.data == 'argument':
            if len(node.children) > 1:
                arg_name = node.children[0].value
            else:
                arg_name = None
            arg_value = do_eval(node.children[-1], stack)
            value = (arg_name, arg_value)
        elif node.data in ['assign_stmt']:
            assert len(node.children) == 3
            dest_node, op_node, src_node = node.children
            #if len(op_node.children) != 1:
            #    raise NotImplementedError('Only `=` operator currently implemented')
            #op = op_node.children[0].value
            op = '='  # FIXME
            assert op == '='
            src_value = do_eval(src_node, stack)
            res = do_eval(dest_node, stack, as_ref=True)
            dest_obj, dest_name = res
            if dest_obj is None:
                stack[-1][dest_name] = src_value
            else:
                setattr(dest_obj, dest_name, src_value)
            value = None
        elif node.data == 'return_stmt':
            value = do_eval(node.children[0], stack)
            raise Return(value)
        elif node.data == 'primary_expr':
            if len(node.children) == 1:
                assert node.children[0].data == 'operand'
                assert len(node.children[0].children) == 1
                kind = node.children[0].children[0].data
                key = do_eval(node.children[0], stack)
                if kind == 'identifier':
                    if as_ref:
                        value = (None, key)
                    else:
                        value = get_stack_item(key, stack)
                else:
                    value = key
            else:
                assert len(node.children) == 2
                obj = do_eval(node.children[0], stack, as_ref=False)
                suffix_node = node.children[1]
                if suffix_node.data == 'dot_suffix':
                    key = do_eval(node.children[1].children[0], stack, as_ref)
                    if as_ref:
                        value = (obj, key)
                    else:
                        value = getattr(obj, key)
                elif suffix_node.data == 'call_suffix':
                    if len(suffix_node.children) != 0:
                        arg_info = do_eval(suffix_node.children[0], stack)
                    else:
                        arg_info = []
                    args = [v for k, v in arg_info if k is None]
                    kwargs = {k: v for k, v in arg_info if k is not None}
                    value = obj(*args, **kwargs)
                elif suffix_node.type == 'slice_suffix':
                    raise NotImplementedError(suffix_node.data)
                else:
                    assert 0, 'unknown suffix_node type: {!r}'.format(suffix_node.data)
        elif node.data == 'def_stmt':
            name_node = node.children[0]
            body_node = node.children[-1]
            name = do_eval(name_node, stack)
            if len(node.children) == 3:
                params_node = node.children[1]
                args_info, min_arg_count, max_arg_count, have_wildargs, have_wildkwargs = get_args_info(params_node, stack)
            else:
                args_info = []
                min_arg_count = 0
                max_arg_count = 0
                have_wildargs = False
                have_wildkwargs = False
            def func(*args, **kwargs):
                assert len(args) >= min_arg_count, '{!r} requires at least {} positional args'.format(name, min_arg_count)
                if max_arg_count is not None:
                    assert len(args) <= max_arg_count, '{!r} requires at most {} positional args'.format(name, max_arg_count)
                frame = {}
                for value, (name, _, _) in zip(args, args_info):
                    frame[name] = value
                stack.append(frame)
                try:
                    return do_eval(body_node, stack)
                except Return as e:
                    return e.value
                finally:
                    stack.pop()
            stack[-1][name] = func
            value = None
        else:
            assert 0, 'unknown node type: {!r}'.format(node.data)
    else:
        assert 0, 'unknown node object type: {!r}'.format(type(node))
    return value

    
source = '''
def make_loop(func):
    def loop():
        func()
        make_loop(func)()
    print(hash(loop))
    return loop


def call_forever(func):
    make_loop(func)()


def my_func():
    print('looping')

call_forever(my_func)
'''

parsed = parser.parse(source)
show(parsed)

global_vars = {
    'print': print,
    'hash': hash,
}
stack = [global_vars]
do_eval(parsed, stack)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0e\xf1\x00\x00\x03c\x08\x06\x00\x00\x00)\xfb\xd7\xd…

-9223372036569187854
looping
-9223372036569390759
looping
285385126
looping
-9223372036569390657
looping
285385194
looping
285624350
looping
-9223372036569390640
looping
-9223372036569390742
looping
-9223372036569728816
looping
-9223372036569390623
looping
285071697
looping
-9223372036569151484
looping
-9223372036569151399
looping
285624452
looping
-9223372036569390606
looping
-9223372036569390691
looping
-9223372036569151382
looping
-9223372036569151433
looping
-9223372036569151331
looping
285624503
looping
-9223372036569151263
looping
-9223372036569142763
looping
285633088
looping
-9223372036569142678
looping
-9223372036569151246
looping
-9223372036569151297
looping
-9223372036569151467
looping
285624418
looping
-9223372036569151348
looping
285385160
looping
-9223372036569142644
looping
285385143
looping
-9223372036569151365
looping
285624333
looping
285385177
looping
-9223372036569151416
looping
285624486
looping
-9223372036569151280
looping
285624469
looping
285624554
looping
28512

RecursionError: maximum recursion depth exceeded in comparison

In [241]:

print(stack)

[{'print': <built-in function print>, 'foo': <function do_eval.<locals>.func at 0x10fe99400>, 'out': 'Hello, world'}]
